In [10]:
import pandas as pd
import re
import numpy as np
import heapq
from tqdm import tqdm
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
pd.set_option('display.max_rows', 500)

In [11]:
segmentdf = pd.read_csv('Pass1EV_Complete.csv')

In [12]:
def strip_string(s):
    return s.strip()

In [28]:
dfp = segmentdf[["Segment", "Model", "Make", "Positives"]]
dfn = segmentdf[["Segment", "Model", "Make", "Negatives"]]
dfw = segmentdf[["Segment", "Model", "Make", "Wishlist"]]
dfep = pd.DataFrame(dfp.assign(Positives = dfp['Positives'].str.split(',')).explode('Positives'))
dfen = pd.DataFrame(dfn.assign(Negatives = dfn['Negatives'].str.split(',')).explode('Negatives'))
dfew = pd.DataFrame(dfw.assign(Wishlist = dfw['Wishlist'].str.split(',')).explode('Wishlist'))
#dfp = dfep[["Segment", "Model", "Make", "Positives"]]
#dfn = dfen[["Segment", "Model", "Make", "Negatives"]]
#dfw = dfew[["Segment", "Model", "Make", "Wishlist"]]
dfepc = dfep.drop(dfep[(dfep['Positives'] == 'None mentioned') | (dfep['Positives'] == 'None') | (dfep['Positives'] == 'N/A') | (dfep['Positives'] == 'None mentioned.') | (dfep['Positives'] == '') | (dfep['Positives'] == 'None') | (dfep['Positives'] == 'None.') | (dfep['Positives'] == ' ') | (dfep['Positives'] == 'No positives mentioned')].index)
dfen['Negatives'] = dfen['Negatives'].apply(strip_string)
#dfen['Negatives'] = dfen['Negatives'].replace('', 'None')
dfenc = dfen.drop(dfen[(dfen['Negatives'] == 'None mentioned') | (dfen['Negatives'] == 'N/A') | (dfen['Negatives'] == 'None mentioned.') | (dfen['Negatives'] == 'None.') | (dfen['Negatives'] == 'No negatives mentioned.')].index)
dfenc['Negatives'] = dfenc['Negatives'].replace(dfenc.head(100).iloc[7]['Negatives'], 'None')
dfenc = dfenc.drop(dfenc[(dfenc['Negatives'] == 'No negatives mentioned') | (dfenc['Negatives'] == '000 miles')].index)
#dfewc = dfew.drop(dfew[(dfew['Positives'] == 'None mentioned') | (dfew['Positives'] == 'None') | (dfew['Positives'] == 'N/A') | (dfep['Positives'] == 'None mentioned.') | (dfep['Positives'] == '') | (dfep['Positives'] == 'None') | (dfep['Positives'] == 'None.') | (dfep['Positives'] == ' ') | (dfep['Positives'] == 'No positives mentioned')].index)
dfewc = dfew.drop(dfew[(dfew['Wishlist'] == 'None mentioned') | (dfew['Wishlist'] == 'None') | (dfew['Wishlist'] == 'N/A') | (dfew['Wishlist'] == 'None mentioned.') | (dfew['Wishlist'] == '') | (dfew['Wishlist'] == 'None') | (dfew['Wishlist'] == 'None.') | (dfew['Wishlist'] == ' ')].index)
master_pos = pd.read_csv('CondensedPositivesEV_L4.csv')
master_neg = pd.read_csv('CondensedNegativesEV_L4.csv')
master_wish = pd.read_csv('CondensedWishlistEV_L4.csv')

dfepc = dfepc.reset_index()
dfepc.drop('index', axis = 1, inplace = True)
dfenc = dfenc.reset_index()
dfenc.drop('index', axis = 1, inplace = True)
dfewc = dfewc.reset_index()
dfewc.drop('index', axis = 1, inplace = True)

In [29]:
#Similarity Analysis
model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('all-mpnet-base-v2')
#n = 2
tags = []
#def similarity_analysis(comment_category, master_list):
for i in tqdm(range(len(dfenc["Negatives"]))):
    sentences = [dfenc["Negatives"].tolist()[i]] + master_neg["Features"].to_list()
    sentence_vectors = model.encode(sentences)
    similarity_score = cosine_similarity([sentence_vectors[0]], sentence_vectors[1:])[0]
    tags.append(master_neg.iloc[similarity_score.tolist().index(max(similarity_score))]["Features"])

100%|████████████████████████████████████████████████████████████████████████████| 43288/43288 [32:15<00:00, 22.36it/s]


In [30]:
df = pd.DataFrame(tags, columns = ["Tags"])
df.to_csv('NegTagsEV.csv')

In [31]:
model = SentenceTransformer('all-MiniLM-L6-v2')
#n = 2
tags = []
for i in tqdm(range(len(dfewc["Wishlist"]))):
    sentences = [dfewc["Wishlist"].tolist()[i]] + master_wish["Features"].to_list()
    sentence_vectors = model.encode(sentences)
    similarity_score = cosine_similarity([sentence_vectors[0]], sentence_vectors[1:])[0]
    tags.append(master_wish.iloc[similarity_score.tolist().index(max(similarity_score))]["Features"])

100%|████████████████████████████████████████████████████████████████████████████| 11578/11578 [27:19<00:00,  7.06it/s]


In [32]:
df = pd.DataFrame(tags, columns = ["Tags"])
df.to_csv('WishTagsEV.csv')

In [33]:
#Similarity Analysis
model = SentenceTransformer('all-MiniLM-L6-v2')
#model = SentenceTransformer('all-mpnet-base-v2')
#n = 2
tags = []
#def similarity_analysis(comment_category, master_list):
for i in tqdm(range(len(dfepc["Positives"]))):
    sentences = [dfepc["Positives"].tolist()[i]] + master_pos["Features"].to_list()
    sentence_vectors = model.encode(sentences)
    similarity_score = cosine_similarity([sentence_vectors[0]], sentence_vectors[1:])[0]
    tags.append(master_pos.iloc[similarity_score.tolist().index(max(similarity_score))]["Features"])

100%|████████████████████████████████████████████████████████████████████████████| 24612/24612 [47:24<00:00,  8.65it/s]


In [34]:
df = pd.DataFrame(tags, columns = ["Tags"])
df.to_csv('PosTagsEV.csv')

In [48]:
#Positve Sentence Similarity took much longer and hence were done in batches across multiple Google Colab instances
dfpos = pd.read_csv('PosTagsEV.csv')
dfpos.drop(['Unnamed: 0'], axis = 1, inplace = True)
dfneg = pd.read_csv('NegTagsEV.csv')
dfneg.drop('Unnamed: 0', axis = 1, inplace = True)
dfwish = pd.read_csv('WishTagsEV.csv')
dfwish.drop('Unnamed: 0', axis = 1, inplace = True)

In [49]:
dfepc['Features'] = dfpos['Tags']
dfenc['Features'] = dfneg['Tags']
dfewc['Features'] = dfwish['Tags']

In [68]:
#Make changes 
#Add Comfort, Handling tag 
dfenc = dfenc.drop(dfenc[(dfenc['Features'] == 'None') | (dfenc['Features'] == 'No negatives mentioned') | (dfenc['Features'] == 'No relevant feedback')].index)
dfewc = dfewc.drop(dfewc[(dfewc['Features'] == 'None') | (dfewc['Features'] == 'No specific wishlist mentioned') | (dfewc['Features'] == 'No relevant feedback')].index)
dfepc = dfepc.drop(dfepc[(dfepc['Features'] == 'None') | (dfepc['Features'] == 'No specific positives mentioned') | (dfepc['Features'] == 'No relevant feedback')].index)
#dfepc.loc[dfepc['Features'] == "No relevant feedback"].head(100)

In [69]:
dfnepc = dfepc.loc[(dfepc["Make"] == "Nissan")]
dfnepc = dfnepc.loc[(dfnepc['Segment'] == "Non-Luxury")]
dfnenc = dfenc.loc[(dfenc["Make"] == "Nissan")]
dfnenc = dfnenc.loc[(dfnenc['Segment'] == "Non-Luxury")]
#dfnewc = dfewc.loc[(dfewc["Make"] == "Nissan")]
#dfnewc = dfnewc.loc[(dfnewc['Segment'] == "Lower Midsize") | (dfnewc['Segment'] == "Compact") | (dfnewc['Segment'] == "Competitive Sports / Performance")]

dftepc = dfepc.loc[(dfepc["Make"] == "Hyundai")]
dftepc = dftepc.loc[(dftepc['Segment'] == "Non-Luxury")]
dftenc = dfenc.loc[(dfenc["Make"] == "Hyundai")]
dftenc = dftenc.loc[(dftenc['Segment'] == "Non-Luxury")]


tab = pd.DataFrame(dfnepc['Features'].value_counts()).reset_index()
tab = tab.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
tab['Features'] = tab['Features'].apply(strip_string)

tab2 = pd.DataFrame(dfnenc['Features'].value_counts()).reset_index()
tab2 = tab2.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
tab2['Features'] = tab2['Features'].apply(strip_string)

#tab3 = pd.DataFrame(dfnewc['Features'].value_counts()).reset_index()
#tab3 = tab3.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
#tab3['Features'] = tab3['Features'].apply(strip_string)

tab4 = pd.DataFrame(dftepc['Features'].value_counts()).reset_index()
tab4 = tab4.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
tab4['Features'] = tab4['Features'].apply(strip_string)

tab5 = pd.DataFrame(dftenc['Features'].value_counts()).reset_index()
tab5 = tab5.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
tab5['Features'] = tab5['Features'].apply(strip_string)

#tab6 = pd.DataFrame(dftewc['Features'].value_counts()).reset_index()
#tab6 = tab6.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
#tab6['Features'] = tab6['Features'].apply(strip_string)

In [88]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(
    header=dict(values=list(tab.columns),
                #fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tab.Features, tab.Frequency],
               #fill_color='lavender',
               align='left'))
])

fig.show()



In [89]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Table(
    header=dict(values=list(tab4.columns),
                #fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tab4.Features, tab4.Frequency],
               #fill_color='lavender',
               align='left'))
])

fig.show()



In [90]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(tab2.columns),
                #fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tab2.Features, tab2.Frequency],
               #fill_color='lavender',
               align='left'))
])

fig.show()

In [91]:
fig = go.Figure(data=[go.Table(
    header=dict(values=list(tab5.columns),
                #fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[tab5.Features, tab5.Frequency],
               #fill_color='lavender',
               align='left'))
])

fig.show()

In [87]:
dfnepc = dfepc.loc[(dfepc["Make"] == "Tesla")]
#dfnepc = dfnepc.loc[(dfnepc['Segment'] == "Non-Luxury")]
dfnenc = dfenc.loc[(dfenc["Make"] == "Tesla")]
#dfnenc = dfnenc.loc[(dfnenc['Segment'] == "Non-Luxury")]
#dfnewc = dfewc.loc[(dfewc["Make"] == "Nissan")]
#dfnewc = dfnewc.loc[(dfnewc['Segment'] == "Lower Midsize") | (dfnewc['Segment'] == "Compact") | (dfnewc['Segment'] == "Competitive Sports / Performance")]

dftepc = dfepc.loc[(dfepc["Make"] == "Lucid")]
#dftepc = dftepc.loc[(dftepc['Segment'] == "Non-Luxury")]
dftenc = dfenc.loc[(dfenc["Make"] == "Lucid")]
#dftenc = dftenc.loc[(dftenc['Segment'] == "Non-Luxury")]


tab = pd.DataFrame(dfnepc['Features'].value_counts()).reset_index()
tab = tab.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
tab['Features'] = tab['Features'].apply(strip_string)

tab2 = pd.DataFrame(dfnenc['Features'].value_counts()).reset_index()
tab2 = tab2.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
tab2['Features'] = tab2['Features'].apply(strip_string)

#tab3 = pd.DataFrame(dfnewc['Features'].value_counts()).reset_index()
#tab3 = tab3.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
#tab3['Features'] = tab3['Features'].apply(strip_string)

tab4 = pd.DataFrame(dftepc['Features'].value_counts()).reset_index()
tab4 = tab4.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
tab4['Features'] = tab4['Features'].apply(strip_string)

tab5 = pd.DataFrame(dftenc['Features'].value_counts()).reset_index()
tab5 = tab5.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
tab5['Features'] = tab5['Features'].apply(strip_string)

#tab6 = pd.DataFrame(dftewc['Features'].value_counts()).reset_index()
#tab6 = tab6.rename(columns = {'index' : 'Features', 'Features' : 'Frequency'})
#tab6['Features'] = tab6['Features'].apply(strip_string)